# Train rt-detr to detect weeds in potato fields

In this notebook we'll train a [rt-detr](https://docs.ultralytics.com/models/rtdetr/) model to detect weeds in a potato fields.
The target weeds are:

- kikuyo
- lengua de vaca
- diente de león

These weeds are common in potatoes fields. The images was captured in 9-10 meters height with a drone. Then, the images was selected and cropped
to 1920x1920 pixels. Finally, the images was labeled with Roboflow.


## Install dependencies

First of all, we'll check the GPU, and install the dependencies.


In [ ]:
!python --version
!cat /etc/os-release

In [ ]:
!pip install -q coco-eval pycocotools ultralytics
!pip install -q wandb ipywidgets --upgrade

## Initialize weights & biases

We'll use weights & biases to log the training process.


In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient

project_name = '02-optimizers-rt-detr-potatoes'
name = 'train-nadam'
user_secrets = UserSecretsClient()
key = user_secrets.get_secret('wandb')


wandb.login(key=key)
wandb.init(
    project=project_name,
    name=name,
    job_type='training',
    tags=[project_name, 'nadam', 'dataset-v4']
)

## Create data.yaml file
We'll create a data.yaml file with the classes and the paths to the images.

In [ ]:
import yaml

base_config = {
    'path': '/path/to/dataset',
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'names': {
        0: 'class_a',
        1: 'class_b',
        2: 'class_c'
    },
}

def create_data_file(data: dict = None, filename: str = 'data.yaml') -> str:
    '''Create custom data.yaml file for training

    Args:
        data (dict): Data to write to configuration file
        filename (str): Name of the configuration file

    Returns:
        str: Name of the configuration file
    '''
    data = data or {}

    # Set default values if not provided
    data['path'] = data['path'] if 'path' in data else base_config['path']
    data['train'] = data['train'] if 'train' in data else base_config['train']
    data['val'] = data['val'] if 'val' in data else base_config['val']
    data['test'] = data['test'] if 'test' in data else base_config['test']
    data['names'] = data['names'] if 'names' in data else base_config['names']

    # Create configuration file
    with open(filename, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

    return filename

### Generate custom data.yaml file

In [ ]:
data = {
    'path': '/kaggle/input/potatoes-weeds-dataset/potatoes-dataset.v2.yolov8',
    'names': {
        0: 'diente_leon',
        1: 'kikuyo',
        2: 'lengua_vaca',
        3: 'otros',
        4: 'papa',
    },
}

create_data_file(data)

## Load default training settings 
We'll use the following default settings obtained from the [ultralytics repository](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/default.yaml).

In [ ]:
default_config = {
    # Train settings
    'imgsz': 640,
    'epochs': 100,
    'patience': 100,
    'batch': 16,
    'optimizer': 'auto',
    'cos_lr': False,
    'close_mosaic': 10,
    'save': True,
    'verbose': True,
    'dropout': 0.0,

    # Val/Test settings
    'val': True,
    'split': 'val',
    'save_json': True,
    'plots': True,

    # Hyperparameters
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 7.5,
    'cls': 0.5,
    'nbs': 64,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 0.0,
    'translate': 0.1,
    'scale': 0.5,
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,
    'fliplr': 0.5,
    'bgr': 0.0,
    'mosaic': 1.0,
    'mixup': 0.0,
    'copy_paste': 0.0,
    'auto_augment': 'randaugment'
}


Now, we'll modify the settings to fit our needs.

In [ ]:
params = {
    **default_config,
    'epochs': 100,
    'imgsz': 640,
    'patience': 20,
    'batch': 8,

    # Hyperparameters
    'optimizer': 'NAdam',
    'cos_lr': False,
    'dropout': 0.0,
    'lr0': 0.001111,
    'weight_decay': 0.0005,
    'momentum': 0.9,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.0,
    'degrees': 0.0
}


## Prepare model
We'll download the rt-detr model and prepare it before training.

In [ ]:
from ultralytics import RTDETR

model = RTDETR('rtdetr-l.pt')

### Visualize the model information

In [ ]:
model.info()

## Train the model

We'll train the baseline model with the potato dataset.


In [ ]:
model.train(
    project=project_name, name=name, data='data.yaml', device=[0, 1], **params
)

## Validate the model
Save validation results to JSON and finish wandb logger.

In [ ]:
metrics = model.val(save_json=True)

wandb.finish()